In [ ]:
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
from IPython.display import Audio, display

def hann_window(n):
    return 0.5 - 0.5 * np.cos(2 * np.pi * np.arange(n) / n)

def pitch_ratio(semitones):
    return 2 ** (semitones / 12.0)

In [ ]:
# Load audio
audio, sr = sf.read("../../resources/drums1.wav")
if audio.ndim > 1:
    audio = audio.mean(axis=1)


In [ ]:
# | UI Label     | Engine Param     | Notes                         |
# | ------------ | ---------------- | ----------------------------- |
# | **Rate**     | grain_rate_hz    | Often nonlinear (log scale)   |
# | **Size**     | grain_size_ms    | Often linkable to Rate        |
# | **Spray**    | spray_ms         | Random offset behind playhead |
# | **Reverse**  | reverse_prob     | Usually 0–100%                |
# | **Pitch**    | pitch_semitones  | 0 = normal                    |
# | **Detune**   | detune_semitones | random jitter                 |
# | **Width**    | width            | Spread; ideally 0–100%        |
# | **Feedback** | feedback         | Grain delay feedback          |

grain_rate_hz        = 30.0        # grains per second
grain_size_ms        = 75.0       # grain length
spray_ms             = 15.0        # random delay behind playhead
reverse_prob         = 0.25        # % reverse grains
pitch_semitones      = 0.0
detune_semitones     = 0.2
width                = 0.0         # 0–2 range
feedback             = 0.35
duration_seconds     = 5.0

# Derived
grain_period = int(sr / grain_rate_hz)
grain_size   = int(sr * grain_size_ms / 1000)
total_samples = int(sr * duration_seconds)

In [ ]:
class Grain:
    def __init__(self, start, reverse, pitch, pan):
        self.start = start
        self.reverse = reverse
        self.pitch = pitch
        self.pan = pan
        self.env = hann_window(grain_size)
        self.phase = 0
        self.length = grain_size

In [ ]:
def schedule_grain(playhead):
    spray_samples = int(np.random.uniform(0, spray_ms) * sr / 1000)
    reverse = np.random.rand() < reverse_prob

    pitch = pitch_ratio(
        pitch_semitones + np.random.uniform(-detune_semitones, detune_semitones)
    )

    if pitch > 1.0:
        spray_samples = max(spray_samples, int(grain_size * (pitch - 1)))

    start = playhead - spray_samples
    start = max(grain_size, start)

    pan = np.random.uniform(-width, width)
    pan = np.clip(pan, -1, 1)

    return Grain(start, reverse, pitch, pan)


In [ ]:
output = np.zeros((2, total_samples))
delay_buffer = np.zeros(total_samples)

active_grains = []
playhead = grain_size
next_grain = 0

for t in range(total_samples - grain_size):

    # --- schedule grains ---
    if t >= next_grain:
        active_grains.append(schedule_grain(playhead))
        next_grain += grain_period

    sample_l = 0.0
    sample_r = 0.0

    still_active = []

    for g in active_grains:
        src_idx = (
            g.start - int(g.phase * g.pitch)
            if g.reverse
            else g.start + int(g.phase * g.pitch)
        )

        if 0 <= src_idx < len(audio) and g.phase < g.length:
            s = audio[src_idx] * g.env[g.phase]

            left  = s * (1 - max(0, g.pan))
            right = s * (1 + min(0, g.pan))

            sample_l += left
            sample_r += right

            g.phase += 1
            still_active.append(g)

    active_grains = still_active

    # --- grain delay feedback ---
    delayed = delay_buffer[t]
    sample_l += delayed
    sample_r += delayed
    delay_buffer[t + grain_size] += (sample_l + sample_r) * 0.5 * feedback

    output[0, t] = sample_l
    output[1, t] = sample_r

    playhead += 1


In [ ]:

output /= np.max(np.abs(output) + 1e-6)

print("Samples:", len(audio), "Sample Rate:", sr)
print(f"Original:")
display(Audio(audio, rate=sr))

print("Processed:")
display(Audio(output, rate=sr))

plt.figure(figsize=(10, 3))
plt.plot(output[0][:20000], label="L")
plt.plot(output[1][:20000], alpha=0.7, label="R")
plt.legend()
plt.title("Granular Output (First ~0.5s)")
plt.show()